<a href="https://colab.research.google.com/github/Anum1228/rl-for-nlp/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
df = pd.read_csv('/content/singapore_airlines_reviews.csv')

needed_columns = ['rating', 'text']
df_final = df[needed_columns]

df_final.head()

In [ ]:
rating_column = df['rating']
review_column = df['text']
# train_rating, test_rating, train_review, test_review = train_test_split(rating_column, review_column, test_size=0.2, random_state=50)

In [ ]:
stop_words = set(stopwords.words('english'))


def preprocessing_data(rev):
    tokens = word_tokenize(rev.lower())
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words and not word.isdigit()]
    return ' '.join(filtered_tokens)

reviews_preprocessed = [preprocessing_data(review) for review in review_column]


# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(max_features=3000)
# train_review_tfidf = vectorizer.fit_transform(train_review)
# test_review_tfidf = vectorizer.transform(test_review)

In [ ]:
train_rating, test_rating, train_review, test_review = train_test_split(rating_column, review_column, test_size=0.2, random_state=50)
print(reviews_preprocessed)


In [ ]:
#use reviews_preprocessed everywhere now
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam

In [ ]:
max_words = 10000
max_len = 150

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(reviews_preprocessed)

X_sequences = tokenizer.texts_to_sequences(reviews_preprocessed)
X_padded = pad_sequences(X_sequences, maxlen=max_len)

X_train, X_test, y_train, y_test = train_test_split(X_padded, rating_column, test_size=0.2, random_state=20)

print(X_sequences[0])
print(X_padded[0])

In [ ]:
# making neural network model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=64, input_length=max_len))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

adam_optim = Adam(learning_rate=0.001)

model.compile(optimizer=adam_optim, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=5, batch_size=16, validation_split=0.2)
# history = history-1


In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(y_pred_classes[:10])

correct_predictions = 0
total_predictions = len(y_test)

for i in range(total_predictions):
    if y_pred[i] == y_test[i]:
        correct_predictions += 1

accuracy = correct_predictions / total_predictions

print(accuracy*100)